<a href="https://colab.research.google.com/github/KimByounghan/Android/blob/master/problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pwd

In [ ]:
!ls -lha

In [ ]:
cd drive/MyDrive/Colab

In [ ]:
!cat /etc/issue

In [ ]:
!cat /proc/cpuinfo

In [ ]:
!cat /proc/meminfo
!cat /proc/cpuinfo
!pwd

In [ ]:
cd MyDrive/Colab

In [ ]:
!pwd

In [ ]:
!ls -lah

**데이콘 경진대회 1등송루션 04장**

상점 신용카드 매출 예측 경진대회를 소개하고 대회를 주최한 동기를 살펴봅니다.

상점 신용카드 매출 예측 경진대회는 2016.6.1.부터 2019.02.28. 까지의 신용카드 매출 데이터를 가지고 2019.3.1부터 2019.5.31.까지 상점별 3개월 총매출을 예측하는 경진대회입니다. 3,4,5월은 봄이고 새 학기 시작입니다. 황사 등 날씨 영향도 받을 수 잇고 5월 가정의 달의 영향도 받을 수 있습니다.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
train = pd.read_csv('./funda_train.csv')
train = train
train.head()

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train[:5]

In [ ]:
train[2:]

In [ ]:
train[train['amount']<0]

In [ ]:
!cat /proc/meminfo

In [ ]:
!pwd

In [ ]:
plt.figure(figsize=(10, 8))
plt.bar(train.columns, train.isnull().sum())
plt.xticks(rotation=90)

결측치가 존재하는 region, type_of_business를 제거하고 결과를 출력합니다.
제거하는 방법은 'drop'명령어를 사용합니다.

In [ ]:
train = train.drop(['region','type_of_business'],axis=1)
train.head()

In [ ]:
train[0:]

In [ ]:
plt.figure(figsize=(32,6))
sns.boxplot(train['amount'])

매출액이 음수로 집계된 경우는 카드 거래 데이터에서 환불이 발생한 것입니다. 우선 환불 발생 현황을 확인하기 우해 0 미만의 행을 출력합니다.

In [ ]:
train[train['amount']<0]

In [ ]:
!cat /proc/meminfo

매출액이 음수로 집계된 카드 결제 내역 노이즈를 제거하기 위한 작업입니다.

환불 발생 시간 이전의 걸내 내역 중 음수 매출액의 절댓값과 같은 행은 제거합니다.
- 환불발생일과 환불발생시간 이전 데이터 중에서 카드 아이디가 같고 음수 매출액의 절댓값과 양수 매출액이 같은 거래 중 최근 거래를 찾아서 해당 인덱스를 제거합니다.

위 거래를 해결하기 위한 함수를 생성합니다

함수는 store_amount_prediction.ipynb로 작업합니다.


In [ ]:
# 거래일과 거래 시간을 합친 변수를 생성합니다.
train['datetime'] = pd.to_datetime(train.transacted_date + " " + train.transacted_time, format='%Y-%m-%d %H:%M:%S')

## 환불 거래를 제거하는 함수를 정의합니다.
def remove_refund(df):
    refund = df[df['amount']<0] # 매출액 음수값 데이터를 추출합니다.
    non_refund = df[df['amount']>0] # 양수값 데이터를 추출합니다
    removed_data = pd.DataFrame()
    for i in tqdm(df.store_id.unique()):
        # 매출액이 양수값인 데이터를 상점별로 나눕니다.
        divided_data = non_refund[non_refund['store_id'] ==i]
        # 매출액이 음수값인 데이터를 상점별로 나눕니다.
        divided_data2 = refund[refund['store_id']==i]

        for neg in divided_data2.to_records()[:]: # 환불 데이터를 차례대로 검사합니다.
            refund_store = neg['store_id']
            refund_id = neg['card_id'] # 환불 카드 아이디 추출
            refund_datetime = neg['datetime'] # 환불 시간을 추출
            refund_amount = abs(neg['amount']) # 매출 음수값의 절대값 구함

            # 환불 시간 이전의 데이터 중 카드 아이디와 환불액이 같은 후보 리스틀 뽐음
            refund_pay_list = divided_data[divided_data['datetime']<=refund_datetime]
            refund_pay_list = refund_pay_list[refund_pay_list['card_id']==refund_id]
            refund_pay_list = refund_pay_list[refund_pay_list['amount']==refund_amount]

            # 후보 리스트가 있으면 카드 아이디와 환불액이 같으면서 최근시간을 제거
            if len(refund_pay_list) != 0:
                # 가장 최근 시간을 구함
                refund_datetime = max(refund_pay_list['datetime'])
                # 가장 최근 시간
                noise_list = divided_data[divided_data['datetime']==refund_datetime]
                # 환불카드 아이디
                noise_list = noise_list[noise_list['card_id']==refund_id]
                # 환불액
                noise_list = noise_list[noise_list['amount']==refund_amount]
                # 인덱스를 통해 제거
                divided_data = divided_data.drop(index=noise_list.index)

        ## 제거한 데이터를 데이터 프레임에 추가
    removed_data = pd.concat([removed_data,divided_data],axis=0)

    return removed_data #들여쓰기 조심

In [ ]:
positive_data = remove_refund(train)

In [ ]:
!cat /proc/meminfo

In [ ]:
positive_data

In [ ]:
plt.figure(figsize=(8, 4))
sns.boxplot(positive_data['amount'])



```
# 코드로 형식 지정됨
```

다운 샘플링(287페이지)

시계열 데이터에서 시간 간격을 넓게 재조정해 데이터 샘플 수를 줄이는 것을 말합니다. 대회에서 요구하는 것은 향후 3개월 동안 상점 매출의 총합을 구하는 것이 문제입니다. 시간 간격을 좁게 해 시계열 모델ㅇㄹ 만들면 예측해야 하는 구간이 커져 불확성이 증가합니다. 따라서 다운 샘플링 과정을 통해 샘플 수를 줄이고 예측 구간을 작게 해서 불확실성을 줄여야 합니다. 

다음은 주어진 데이터에서 월 단위로 다운 샘플링한 예시입니다.

In [ ]:
positive_data.head(20)

데이터 시간 함수를 월 단위로 재조정하는 함수

In [ ]:
## 월 단위 다운 샘플링 함수를 정의
def month_resampling(df):
    new_data = pd.DataFrame()
    # 년도와 월을 합친 변수를 생성
    df['year_month'] = df['transacted_date'].str.slice(stop=7)
    # 데이터의 전체 기간을 추출
    year_month = df['year_month'].drop_duplicates()
    # 상점 아이디별로 월 단위 매출액 총합을 구함
    downsampling_data = df.groupby(['store_id', 'year_month']).amount.sum()
    downsampling_data = pd.DataFrame(downsampling_data)
    downsampling_data = downsampling_data.reset_index(drop=False, inplace=False)

    for i in tqdm(df.store_id.unique()):
        # 상점별로 데이터를 처리
        store = downsampling_data[downsampling_data['store_id']==i]
        # 각 상점의 처음 매출이 발생한 월을 구함
        start_time = min(store['year_month'])
        # 모든 상점을 전체 기간 데이터로 만듬
        store = store.merge(year_month, how='outer')
        # 데이터를 시간순으로 정렬
        store = store.sort_values(by=['year_month'], axis=0, ascending=True)
        # 매출이 발생하지 않는 월은 2로 채움
        store['amount'] = store['amount'].fillna(2)
        # 상점 아이디 결측치를 채움
        store['store_id'] = store['store_id'].fillna(i)
        # 처음 매출이 발생한 월 이후만 뽑음
        store = store[store['year_month']>=start_time]

        new_data = pd.concat([new_data,store], axis=0)
    
    return new_data


In [ ]:
resampling_data = month_resampling(positive_data)
resampling_data['store_id'] = resampling_data['store_id'].astype(int)
resampling_data

**날짜 지정 밤위 생성과 시리즈 객체 변환(291쪽)**

시계열 데이터 분석을 위해서는 데이터프레임을 시리즈 객체로 변환하여야 합니다. 

In [ ]:
# 데이터 타입 출력
print(type(resampling_data))

In [ ]:
# 데이터 프레임을 Series로 변환하는 함수
def time_series(df, i):
    # 상점별로 데이터를 뽑음
    store = df[df['store_id']==i]
    ## 날짜 지정 범위는 영업 시작 월부터 2019년 3월 전까지 영업 마감일 기준
    index = pd.date_range(min(store['year_month']), '2019-03', freq='BM')
    ## 시리즈 객체로 변환
    ts = pd.Series(store['amount'].values, index=index)
    return ts

In [ ]:
# 상점 아이디가 0번인 데이터를 시리즈 객체로 변환
store_0 = time_series(resampling_data, 0)
store_0

In [ ]:
!cat /proc/meminfo

In [ ]:
from google.colab import files

In [ ]:
files.download('problem.ipynb')

In [ ]:
upload = files.upload()

In [ ]:
!ls -lh

In [ ]:
!echo $UID